In [1]:
import polars as pd

In [2]:
processed_data = pd.read_ndjson('../checkpoint-1/processed_data.json')

processed_data.head()

category,rating,label,text_,text_processed,text_tokenized,text_embeddings
str,f64,str,str,str,list[str],list[f64]
"""Home and Kitchen""",5.0,"""CG""","""Love this! Well made, sturdy,…","""love well made sturdy comforta…","[""love"", ""well"", … ""pretty""]","[-0.065527, 0.00191, … 0.027523]"
"""Home and Kitchen""",5.0,"""CG""","""love it, a great upgrade from …","""love great upgrade original iv…","[""love"", ""great"", … ""years""]","[-0.119937, 0.089622, … 0.018372]"
"""Home and Kitchen""",5.0,"""CG""","""This pillow saved my back. I l…","""pillow saved back love look fe…","[""pillow"", ""saved"", … ""pillow""]","[-0.053175, 0.018608, … -0.020651]"
"""Home and Kitchen""",1.0,"""CG""","""Missing information on how to …","""missing information use great …","[""missing"", ""information"", … ""price""]","[0.007157, 0.063695, … 0.061616]"
"""Home and Kitchen""",5.0,"""CG""","""Very nice set. Good quality. W…","""nice set good quality set two …","[""nice"", ""set"", … ""months""]","[-0.036983, 0.011753, … 0.075562]"


In [3]:
processed_data = processed_data.drop_nulls()

In [4]:
processed_data.null_count()

category,rating,label,text_,text_processed,text_tokenized,text_embeddings
u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0


In [5]:
X = processed_data['text_embeddings'].to_list()
y = processed_data['label'].to_list()

len(X), len(y)

(40408, 40408)

In [6]:
# base preprocessing
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

y = lb.fit_transform(y).flatten()

In [7]:
y = y.flatten()

In [8]:
len(X), len(y)

(40408, 40408)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [80]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(X_train, y_train)

y_preds_nb = gnb.predict(X_test)

In [77]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

y_preds_logreg = logreg.predict(X_test)

In [82]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)

y_preds_rfc = rfc.predict(X_test)

In [79]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_test, y_pred=y_preds_logreg))

              precision    recall  f1-score   support

           0       0.58      0.56      0.57      6060
           1       0.58      0.60      0.59      6063

    accuracy                           0.58     12123
   macro avg       0.58      0.58      0.58     12123
weighted avg       0.58      0.58      0.58     12123



In [81]:
print(classification_report(y_true=y_test, y_pred=y_preds_nb))

              precision    recall  f1-score   support

           0       0.56      0.56      0.56      6060
           1       0.56      0.57      0.57      6063

    accuracy                           0.56     12123
   macro avg       0.56      0.56      0.56     12123
weighted avg       0.56      0.56      0.56     12123



In [83]:
print(classification_report(y_true=y_test, y_pred=y_preds_rfc))

              precision    recall  f1-score   support

           0       0.57      0.58      0.57      6060
           1       0.57      0.57      0.57      6063

    accuracy                           0.57     12123
   macro avg       0.57      0.57      0.57     12123
weighted avg       0.57      0.57      0.57     12123



Hyperparameter Tuning Logistic Regression

In [87]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2', 'l1'],  # Ensure solver supports chosen penalties
    'solver': ['liblinear', 'saga'],
    'class_weight': [None, 'balanced']
}

logreg = LogisticRegression()

gs = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5, scoring='accuracy')

gs.fit(X_train, y_train)

print("Best Hyperparameters:", gs.best_params_)

Best Hyperparameters: {'C': 10, 'class_weight': None, 'penalty': 'l1', 'solver': 'liblinear'}


In [88]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C=10, class_weight=None, penalty='l1', solver='liblinear')

logreg.fit(X_train, y_train)

y_preds_logreg = logreg.predict(X_test)

print(classification_report(y_true=y_test, y_pred=y_preds_logreg))

              precision    recall  f1-score   support

           0       0.59      0.57      0.58      6060
           1       0.58      0.60      0.59      6063

    accuracy                           0.59     12123
   macro avg       0.59      0.59      0.59     12123
weighted avg       0.59      0.59      0.59     12123



In [11]:
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500)

mlp.fit(X_train, y_train)

y_preds_nlp = mlp.predict(X_test)

print(accuracy_score(y_true=y_test, y_pred=y_preds_nlp))

0.577497319145426
